# Lab: Bring your own custom container with Amazon SageMaker

## Overview
Here, we’ll show how to package a simple Python example which showcases the decision tree algorithm from the widely used scikit-learn machine learning package. The example is purposefully fairly trivial since the point is to show the surrounding structure that you’ll want to add to your own code so you can train and host it in Amazon SageMaker.

The ideas shown here will work in any language or environment. You’ll need to choose the right tools for your environment to serve HTTP requests for inference, but good HTTP environments are available in every language these days.

In this example, we use a single image to support training and hosting. This is easy because it means that we only need to manage one image and we can set it up to do everything. Sometimes you’ll want separate images for training and hosting because they have different requirements. Just separate the parts discussed below into separate Dockerfiles and build two images. Choosing whether to have a single image or two images is really a matter of which is more convenient for you to develop and manage.

If you’re only using Amazon SageMaker for training or hosting, but not both, there is no need to build the unused functionality into your container.

## Building the container
Docker provides a simple way to package arbitrary code into an image that is totally self-contained. Once you have an image, you can use Docker to run a container based on that image. Running a container is just like running a program on the machine except that the container creates a fully self-contained environment for the program to run. Containers are isolated from each other and from the host environment, so the way you set up your program is the way it runs, no matter where you run it.

Amazon SageMaker uses Docker to allow users to train and deploy arbitrary algorithms.

### Parts of the container
In the container directory are all the components you need to package the sample algorithm for Amazon SageMager:

```
.
|-- Dockerfile
|-- build_and_push.sh
`-- decision_trees
    |-- nginx.conf
    |-- predictor.py
    |-- serve
    |-- train
    `-- wsgi.py
```

Let’s discuss each of these in turn:

- Dockerfile describes how to build your Docker container image. More details below.
- build_and_push.sh is a script that uses the Dockerfile to build your container images and then pushes it to ECR. We’ll invoke the commands directly later in this notebook, but you can just copy and run the script for your own algorithms.
- decision_trees is the directory which contains the files that will be installed in the container.
- local_test is a directory that shows how to test your new container on any computer that can run Docker, including an Amazon SageMaker notebook instance. Using this method, you can quickly iterate using small datasets to eliminate any structural bugs before you use the container with Amazon SageMaker. We’ll walk through local testing later in this notebook.

In this simple application, we only install five files in the container.

The files that we’ll put in the container are:

- nginx.conf is the configuration file for the nginx front-end. Generally, you should be able to take this file as-is.
- predictor.py is the program that actually implements the Flask web server and the decision tree predictions for this app. You’ll want to customize the actual prediction parts to your application. Since this algorithm is simple, we do all the processing here in this file, but you may choose to have separate files for implementing your custom logic.
- serve is the program started when the container is started for hosting. It simply launches the gunicorn server which runs multiple instances of the Flask app defined in predictor.py. You should be able to take this file as-is.
- train is the program that is invoked when the container is run for training. You will modify this program to implement your training algorithm.
- wsgi.py is a small wrapper used to invoke the Flask app. You should be able to take this file as-is.

In summary, the two files you will probably want to change for your application are train and predictor.py

### The Dockerfile
The Dockerfile describes the image that we want to build. You can think of it as describing the complete operating system installation of the system that you want to run. A Docker container running is quite a bit lighter than a full operating system, however, because it takes advantage of Linux on the host machine for the basic operations.

For the Python science stack, we will start from a standard Ubuntu installation and run the normal tools to install the things needed by scikit-learn. Finally, we add the code that implements our specific algorithm to the container and set up the right environment to run under.

In [1]:
# Cell #1

!curl https://raw.githubusercontent.com/aws-samples/amazon-sagemaker-immersion-day/master/scikit_bring_your_own.zip --output scikit_bring_your_own.zip
!unzip scikit_bring_your_own.zip
!mv scikit_bring_your_own/data ./
!mv scikit_bring_your_own/container ./
!rm scikit_bring_your_own.zip
!rm -rf scikit_bring_your_own
!cat container/Dockerfile

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 41277  100 41277    0     0   108k      0 --:--:-- --:--:-- --:--:--  108k
Archive:  scikit_bring_your_own.zip
   creating: scikit_bring_your_own/
   creating: scikit_bring_your_own/container/
  inflating: scikit_bring_your_own/container/build_and_push.sh  
   creating: scikit_bring_your_own/container/local_test/
  inflating: scikit_bring_your_own/container/local_test/payload.csv  
  inflating: scikit_bring_your_own/container/local_test/train_local.sh  
   creating: scikit_bring_your_own/container/local_test/test_dir/
   creating: scikit_bring_your_own/container/local_test/test_dir/input/
   creating: scikit_bring_your_own/container/local_test/test_dir/input/config/
 extracting: scikit_bring_your_own/container/local_test/test_dir/input/config/hyperparameters.json  
 extracting: scikit_bring_your_own/container/local_test/test_d

## Building and registering the container

In [2]:
%%sh

# Cell #2


# The name of our algorithm
algorithm_name=sagemaker-decision-trees

cd container

chmod +x decision_trees/train
chmod +x decision_trees/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Login Succeeded
Sending build context to Docker daemon  56.32kB
Step 1/9 : FROM ubuntu:16.04
 ---> dfeff22e96ae
Step 2/9 : MAINTAINER Amazon AI <sage-learner@amazon.com>
 ---> Using cache
 ---> 3a496dbd81a3
Step 3/9 : RUN apt-get -y update && apt-get install -y --no-install-recommends          wget          python          nginx          ca-certificates     && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> d222fade3955
Step 4/9 : RUN wget https://bootstrap.pypa.io/get-pip.py && python get-pip.py &&     pip install numpy==1.16.2 scipy==1.2.1 scikit-learn==0.20.2 pandas flask gevent gunicorn &&         (cd /usr/local/lib/python2.7/dist-packages/scipy/.libs; rm *; ln ../../numpy/.libs/* .) &&         rm -rf /root/.cache
 ---> Using cache
 ---> bcd362a222ad
Step 5/9 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> db11a483b729
Step 6/9 : ENV PYTHONDONTWRITEBYTECODE=TRUE
 ---> Using cache
 ---> 8b5901df42f0
Step 7/9 : ENV PATH="/opt/program:${PATH}"
 ---> Using cache
 ---> 5f235e10f

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



## Using the container
Here we specify a bucket to use and the role that will be used for working with SageMaker.

In [3]:
# Cell #3

# S3 prefix
prefix = 'DEMO-scikit-byo-iris'

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

The session remembers our connection parameters to SageMaker. We’ll use it to perform all of our SageMaker operations.

In [4]:
# Cell #4

import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

When training large models with huge amounts of data, you’ll typically use big data tools, like Amazon Athena, AWS Glue, or Amazon EMR, to create your data in S3. For the purposes of this example, we’re using some the classic Iris dataset, which we have included.

We can use use the tools provided by the SageMaker Python SDK to upload the data to a default bucket.

In [5]:
# Cell #5

WORK_DIRECTORY = 'data'

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

In order to use SageMaker to fit our algorithm, we’ll create an Estimator that defines how to use the container to train. This includes the configuration we need to invoke SageMaker training:

- The container name. This is constructed as in the shell commands above.
- The role. As defined above.
- The instance count which is the number of machines to use for training.
- The instance type which is the type of machine to use for training.
- The output path determines where the model artifact will be written.
- The session is the SageMaker session object that we defined above.

Then we use fit() on the estimator to train against the data that we uploaded above.

In [6]:
# Cell #6

account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/sagemaker-decision-trees:latest'.format(account, region)

tree = sage.estimator.Estimator(image,
                       role, train_instance_count=1, train_instance_type='ml.c4.2xlarge',
                       output_path="s3://{}/output".format(sess.default_bucket()),
                       sagemaker_session=sess)

tree.fit(data_location)

Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


2020-10-29 15:53:58 Starting - Starting the training job...
2020-10-29 15:54:00 Starting - Launching requested ML instances......
2020-10-29 15:55:23 Starting - Preparing the instances for training...
2020-10-29 15:55:57 Downloading - Downloading input data...
2020-10-29 15:56:19 Training - Downloading the training image...
2020-10-29 15:56:55 Uploading - Uploading generated training model
2020-10-29 15:56:55 Completed - Training job completed
Starting the training.
Training complete.
Training seconds: 58
Billable seconds: 58


## Hosting your model
You can use a trained model to get real time predictions using HTTP endpoint. Follow these steps to walk you through the process.

Deploying the model to SageMaker hosting just requires a deploy call on the fitted model. This call takes an instance count, instance type, and optionally serializer and deserializer functions. These are used when the resulting predictor is created on the endpoint.

In [7]:
# Cell #7

from sagemaker.predictor import csv_serializer
predictor = tree.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge', serializer=csv_serializer)

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


-------------!

## Preparing test data to run inferences

In order to do some predictions, we’ll extract some of the data we used for training and do predictions against it. This is, of course, bad statistical practice, but a good way to see how the mechanism works.

In [8]:
# Cell #8

shape=pd.read_csv("data/iris.csv", header=None)
shape.sample(3)

,0,1,2,3,4
87,versicolor,6.3,2.3,4.4,1.3
113,virginica,5.7,2.5,5.0,2.0
126,virginica,6.2,2.8,4.8,1.8


In [9]:
# Cell #9

# drop the label column in the training set
shape.drop(shape.columns[[0]],axis=1,inplace=True)
shape.sample(3)

,1,2,3,4
138,6.0,3.0,4.8,1.8
72,6.3,2.5,4.9,1.5
126,6.2,2.8,4.8,1.8


In [10]:
# Cell #10

import itertools

a = [50*i for i in range(3)]
b = [40+i for i in range(10)]
indices = [i+j for i,j in itertools.product(a,b)]

test_data=shape.iloc[indices[:-1]]

## Run predictions

Prediction is as easy as calling predict with the predictor we got back from deploy and the data we want to do predictions with. The serializers take care of doing the data conversions for us.

In [11]:
# Cell #11

print(predictor.predict(test_data.values).decode('utf-8'))

setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
virginica
virginica
virginica
virginica
virginica
virginica
virginica
virginica
virginica



## Cleanup
After completing the lab, use these steps to [delete the endpoint](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-cleanup.html) or run the following code


In [12]:
# Cell #12
sess.delete_endpoint(predictor.endpoint)